In [231]:
import pandas as pd
import numpy as np
import re
from collections import Counter,OrderedDict
import csv
from sklearn.ensemble import RandomForestClassifier
from statistics import mean
import json
from sklearn.model_selection import train_test_split

In [232]:
emp = pd.read_csv('../data/hrdata_OSF.csv')
emp_list = {}
month_dict = {}
for em in emp.to_dict('records'):
    emp_list[em['name']] = {'bank':em['bank'], 'month':em['month'], 'branch':em['branch_id']}

pre_branch = {}
branch = {}
# for em in emp_list:
#     br = emp_list[em]['branch']
#     if br not in branch:
#         branch[br] = 1
#     else:
#         temp = branch[br]
#         branch[br] = temp+1

In [233]:
# contemporaneous

merger = 'm1'

# month 2-24
# individual month
r = []
r_dict = []
for i in range(23):
    X_train = pd.read_csv('../data/traintest/'+'m1_train_'+str(i+2)+'.csv')
    y_train = X_train['bank'];
    X_train = X_train.drop(columns=['bank','sender','month'], axis=1)
    X_test = pd.read_csv('../data/traintest/'+'m1_test_'+str(i+2)+'.csv')
    # dataset = pd.read_csv('../data/'+str(i+2)+'.csv')
    # train, test = train_test_split(dataset, test_size=0.5,random_state=42)

    # X_train = train.drop(columns=['bank','sender','month','n.terms'], axis=1)
    # y_train = train['bank'];
    # X_test = test.drop(columns=['bank','sender','month','n.terms'], axis=1)
    banks = X_test['sender']
    X_test = X_test.drop(columns=['bank','sender','month'], axis=1)

    for b in banks.tolist():
        r_dict.append(b)
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train,y_train.values.ravel())
    x = clf.predict_proba(X_test)
    for e in x:
        r.append(e)

In [234]:
len(r_dict)

4486

In [235]:
len(r)

4486

In [236]:
i = 0

for prediction in r:
    score  = prediction
    emp = r_dict[i]
    if emp_list[emp]['bank'] == 'B':
        if 'diff' not in emp_list[emp]:
            if score[0] > 0.5:
                emp_list[emp]['correct'] = [score[0]]
            emp_list[emp]['diff'] = [score[0]]
            emp_list[emp]['diff_L'] = [score[1]]
        else:
            if score[0] <= 0.5:
                if 'correct' in emp_list[emp]:
                    I =  emp_list[emp]['correct']
                    I.append(score[0])
                    emp_list[emp]['correct'] = I
                else:
                    emp_list[emp]['correct'] = [score[0]]

            B =  emp_list[emp]['diff']
            B.append(score[0])
            emp_list[emp]['diff'] = B

            L =  emp_list[emp]['diff_L']
            L.append(score[1])
            emp_list[emp]['diff_L'] = L
        
    i+=1

In [237]:
# HD
i = 5
pre_1 = pd.read_csv('../data/traintest/'+'m1_train_'+str(i-1)+'.csv')
pre_2 = pd.read_csv('../data/traintest/'+'m1_train_'+str(i-2)+'.csv')
pre_3 = pd.read_csv('../data/traintest/'+'m1_train_'+str(i-3)+'.csv')
# data_1 = pd.read_csv('../data/'+str(i-1)+'.csv')
# data_2 = pd.read_csv('../data/'+str(i-2)+'.csv')
# data_3 = pd.read_csv('../data/'+str(i-3)+'.csv')
# pre_1, pre_1_test = train_test_split(data_1, test_size=0.5,random_state=42)
# pre_2, pre_2_test = train_test_split(data_2, test_size=0.5,random_state=42)
# pre_3, pre_3_test = train_test_split(data_3, test_size=0.5,random_state=42)

frames = [pre_1, pre_2, pre_3]
X = pd.concat(frames)
pre_density = X
y = X['bank']
X = X.drop(columns=['bank','sender','month'], axis=1)
clf = RandomForestClassifier(random_state=42)
clf.fit(X,y.values.ravel())

r2 = []
r2_dict = []
for i in range(19):
    # test = pd.read_csv('../data/traintest/'+'m1_test_'+str(i+6)+'.csv')
    test = pd.read_csv('../data/'+str(i+6)+'.csv')
    data_train,data_test = train_test_split(test, test_size=0.5,random_state=42)
    banks = data_test['sender']
    for b in banks.tolist():
        r2_dict.append(b)
    c = data_test.drop(columns=['bank','sender','month'], axis=1)
    x  = clf.predict_proba(c)
    for e in x:
        r2.append(e)

In [238]:
for s in pre_density.to_dict('records'):
    br = emp_list[s['sender']]['branch']
    print(br)
    if br not in pre_branch:
        pre_branch[br] = 1
    else:
        temp = pre_branch[br]
        pre_branch[br] = temp+1

7
35
10
46
34
2
19
23
13
15
32
23
46
26
22
3
51
46
3
3
19
21
3
36
3
11
3
39
42
15
1
3
16
23
26
10
12
3
3
3
1
49
36
46
3
16
3
19
3
38
14
49
3
3
14
16
35
14
35
8
12
12
5
7
3
3
10
10
19
5
18
19
17
41
4
19
11
4
3
23
8
38
10
39
23
10
36
3
46
2
3
3
3
17
18
47
46
36
8
46
3
24
13
46
10
1
47
3
4
38
19
18
5
1
37
17
10
10
3
3
14
5
3
47
23
5
23
46
35
8
3
19
1
46
8
3
12
46
46
10
3
10
8
16
13
3
3
1
20
16
51
19
7
1
46
46
10
35
36
23
19
34
46
47
10
3
47
3
3
17
46
3
3
32
1
19
3
12
19
3
18
3
32
19
19
21
3
19
10
11
10
3
3
16
11
10
26
10
10
3
3
34
19
7
35
10
46
34
2
19
23
13
15
32
23
46
26
22
3
51
46
3
3
19
21
3
36
3
11
3
39
42
15
1
3
16
23
26
10
12
3
3
3
1
49
36
46
3
16
3
19
3
38
14
49
3
3
14
16
35
14
35
8
12
12
5
7
3
3
10
10
19
5
18
19
17
41
4
19
11
4
3
23
8
38
10
39
23
10
36
3
46
2
3
3
3
17
18
47
46
36
8
46
3
24
13
46
10
1
47
3
4
38
19
18
5
1
37
17
10
10
3
3
14
5
3
47
23
5
23
46
35
8
3
19
1
46
8
3
12
46
46
10
3
10
8
16
13
3
3
1
20
16
51
19
7
1
46
46
10
35
36
23
19
34
46
47
10
3
47
3
3
17
46
3
3
32
1
19

In [239]:
for r in range(len(r2)):
    score = r2[r]
    emp = r2_dict[r]
    
    br = emp_list[emp]['branch']
    if br not in branch:
        branch[br] = 1
    else:
        temp = branch[br]
        branch[br] = temp+1

    if emp_list[emp]['bank'] == 'B':
        if 'B' not in emp_list[emp]:
            if score[0] <= 0.5:
                emp_list[emp]['incorrect'] = [score[0]]
            emp_list[emp]['ret'] = [score[0]]
            emp_list[emp]['ret_L'] = [score[1]]
        else:
            if score[0] <= 0.5:
                if 'incorrect' in emp_list[emp]:
                    I =  emp_list[emp]['incorrect']
                    I.append(score[0])
                    emp_list[emp]['incorrect'] = I
                else:
                    emp_list[emp]['incorrect'] = [score[0]]

            B =  emp_list[emp]['ret']
            B.append(score[0])
            emp_list[emp]['ret'] = B

            L =  emp_list[emp]['ret']
            L.append(score[1])
            emp_list[emp]['ret'] = L

        if 'email' not in emp_list[emp]:
            emp_list[emp]['email'] = 1
        else:
            email =  emp_list[emp]['email'] + 1
            emp_list[emp]['email'] = email

In [240]:
for emp in emp_list:
    if 'diff' in emp_list[emp]:
        diff = emp_list[emp]['diff']
        emp_list[emp]['diff'] = round(mean(diff),2)

        diff_L = emp_list[emp]['diff_L']
        emp_list[emp]['diff_L'] = round(mean(diff_L),2)
    if 'ret' in emp_list[emp]:
        B = emp_list[emp]['ret']
        emp_list[emp]['ret'] = round(mean(B),2)

        L = emp_list[emp]['ret_L']
        emp_list[emp]['ret_L'] = round(mean(L),2)

In [241]:
for emp in emp_list:
    temp = emp_list[emp]
    # if 'ret' in temp and 'diff' in temp and temp['bank'] == 'B':
    #     if temp['ret'] <= 0.5 and temp['diff'] > 0.5:
    #         temp['innov'] = temp['ret']*temp['diff']
    #     else:
    #         temp['innov'] =  0

    if 'incorrect' in temp and 'correct' in temp:
        temp['innov_1'] = round(mean(temp['incorrect']),2) * round(mean(temp['correct']),2)


In [242]:
pre_branch

{7: 9,
 35: 15,
 10: 54,
 46: 48,
 34: 9,
 2: 6,
 19: 48,
 23: 24,
 13: 9,
 15: 6,
 32: 9,
 26: 9,
 22: 3,
 3: 135,
 51: 6,
 21: 6,
 36: 15,
 11: 12,
 39: 6,
 42: 3,
 1: 24,
 16: 18,
 12: 15,
 49: 6,
 38: 9,
 14: 12,
 8: 18,
 5: 15,
 18: 12,
 17: 12,
 41: 3,
 4: 9,
 47: 15,
 24: 3,
 37: 3,
 20: 3}

In [243]:
result = pd.DataFrame(columns=['month','sender','diff','ret','innov','density','n_emails'])

for emp in emp_list:
    temp = emp_list[emp]
    if temp['branch'] in pre_branch:
        pre = pre_branch[temp['branch']]
    else:
        pre = 0
    if temp['branch'] in branch:
        post = branch[temp['branch']]
    else:
        post = 0

    if post!=0:
        d = round(float(pre)/float(post),2)
    else:
        d = 0

    if 'ret' in temp and 'diff' in temp and 'innov_1' in temp:
        d = {'month':temp['month'],'sender':emp,'diff':temp['diff'],'ret':temp['ret'],'innov':temp['innov_1'],'n_emails':temp['email'],'density':d}
        result = result.append(d, ignore_index = True)
    elif 'ret' in temp and 'diff' in temp and 'diff_L' in temp:
        d = {'month':temp['month'],'sender':emp,'diff':temp['diff'],'ret':temp['ret'],'innov':temp['diff_L'],'n_emails':temp['email'],'density':d}
        result = result.append(d, ignore_index = True)
        

/var/folders/tj/yk7dvkjd2_ndkr_0hlr4cgq80000gn/T/ipykernel_22026/129463295.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(d, ignore_index = True)
/var/folders/tj/yk7dvkjd2_ndkr_0hlr4cgq80000gn/T/ipykernel_22026/129463295.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(d, ignore_index = True)
/var/folders/tj/yk7dvkjd2_ndkr_0hlr4cgq80000gn/T/ipykernel_22026/129463295.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(d, ignore_index = True)
/var/folders/tj/yk7dvkjd2_ndkr_0hlr4cgq80000gn/T/ipykernel_22026/129463295.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

In [244]:
# result

result.to_csv('output.csv', sep=',', encoding='utf-8')

# with open("output.json", "w") as outfile:
#     json.dump(result.to_dict('records'), outfile, indent = 4)